<a href="https://colab.research.google.com/github/jaivanti/syracuse/blob/main/espcn_xiph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import libraries

In [ ]:
import tensorflow as tf

import os
import math
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
import cv2
from IPython.display import display
from keras.models import load_model
import PIL

get all values for upscaling

In [ ]:
def get_lowres_image(img, upscale_factor):
  """Return low-resolution image to use as model input."""
  return img.resize((img.size[0] // upscale_factor, img.size[1] // upscale_factor),PIL.Image.BICUBIC)

In [ ]:

def upscale_image(model, img):
    """Predict the result based on input image and restore the image as RGB."""
  
    #ycbcr = img.convert("YCbCr")
    ycbcr = load_img('/content/drive/MyDrive/SRCDN/LR/video14/1.bmp')
    y, cb, cr = ycbcr.split()
    y = img_to_array(y)
    y = y.astype("float32") / 255.0
    
    input = np.expand_dims(y, axis=0)
    out = model.predict(input)


    out_img_y = out[0]
    out_img_y *= 255.0

    # Restore the image in RGB color space.
    out_img_y = out_img_y.clip(0, 255)
    out_img_y = out_img_y.reshape((np.shape(out_img_y)[0], np.shape(out_img_y)[1]))
    out_img_y = PIL.Image.fromarray(np.uint8(out_img_y), mode="L")
    out_img_cb = cb.resize(out_img_y.size, PIL.Image.BICUBIC)
    out_img_cr = cr.resize(out_img_y.size, PIL.Image.BICUBIC)
    out_img = PIL.Image.merge("YCbCr", (out_img_y, out_img_cb, out_img_cr)).convert(
        "RGB"
    )
    return out_img


load model

In [ ]:
#model = load_model("/content/drive/MyDrive/SRCDN/model/network.h5")   #suyash
model = load_model("/content/drive/MyDrive/SRCDN/model/gfgModel.h5")   #jaivanti

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


create directories

In [ ]:
#!mkdir /content/drive/MyDrive/SRCDN/LR/video2
!mkdir /content/drive/MyDrive/SRCDN/LR/video16
%cd /content/drive/MyDrive/SRCDN/LR/video16

/content/drive/.shortcut-targets-by-id/1lZzJclOmbbPx_0_m3KHq80BRx5R6ZSuO/SRCDN/LR/video16


In [ ]:
!mkdir /content/drive/MyDrive/SRCDN/result/video16

mkdir: cannot create directory ‘/content/drive/MyDrive/SRCDN/result/video16’: File exists


load the bus_cif.yuv file from drive with size 352x288

In [ ]:
!ffmpeg -s 352x288 -i /content/bus_cif.yuv %d.bmp

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
im = load_img('/content/drive/MyDrive/SRCDN/result/video15/op1.bmp')
im.size

(1056, 864)

upscale it

In [ ]:
#/content/drive/MyDrive/SRCDN/LR/video1/1.png
upscale_factor = 4
t_sum = 0
for i in range(1, 5):
  #img_path = ('/content/drive/MyDrive/SRCDN/LR/video11/' + str(i) + '.png')
  img_path = ('/content/drive/MyDrive/SRCDN/LR/video14/'  + str(i) + '.bmp')
  img = load_img(img_path)
  lowres_input = get_lowres_image(img, upscale_factor)
  w = lowres_input.size[0] * upscale_factor
  h = lowres_input.size[1] * upscale_factor
  #highres_img = img.resize((w, h))
  start = time.time()
  prediction = upscale_image(model, img)
  end = time.time()
  print(end-start)
  dif = end-start
  t_sum += dif
  prediction.save('/content/drive/MyDrive/SRCDN/result/video15/op' + str(i) + '.bmp')
  
avg = t_sum/16
print("the avg is", avg)



0.12935662269592285
0.11472010612487793
0.12139701843261719
0.11494064331054688
the avg is 0.030025899410247803


see the size


In [ ]:
im = load_img('/content/drive/MyDrive/SRCDN/LR/video14/1.bmp')
im1 = get_lowres_image(im, 3)



In [ ]:
im.size

(352, 288)

In [ ]:
type(im)

PIL.BmpImagePlugin.BmpImageFile

In [ ]:
start = time.time()
#img = load_img('/content/drive/MyDrive/SRCDN/LR/video14/1.bmp')
ycbcr = load_img('/content/drive/MyDrive/SRCDN/LR/video14/1.bmp')
#ycbcr = img.convert("YCbCr")
y, cb, cr = ycbcr.split()
y = img_to_array(y)
y = y.astype("float32") / 255.0
input = np.expand_dims(y, axis=0)
end = time.time()
t1 = end-start
print("t1 is ", t1)
st = time.time()
out = model.predict(input)
en = time.time()
t3 = en-st
print("upscaling inf time is ",t3 )
start1 = time.time()
out_img_y = out[0]
out_img_y *= 255.0

# Restore the image in RGB color space.
out_img_y = out_img_y.clip(0, 255)
out_img_y = out_img_y.reshape((np.shape(out_img_y)[0], np.shape(out_img_y)[1]))
out_img_y = PIL.Image.fromarray(np.uint8(out_img_y), mode="L")
out_img_cb = cb.resize(out_img_y.size, PIL.Image.BICUBIC)
out_img_cr = cr.resize(out_img_y.size, PIL.Image.BICUBIC)
out_img = PIL.Image.merge("YCbCr", (out_img_y, out_img_cb, out_img_cr)).convert(
    "RGB"
)
end1 = time.time()
t2 = end1-start1
print("t2 is ", t2)
print("t1 + t2 is ", t1 + t2)
print("whole is", t1 + t2 + t3)





t1 is  0.005537509918212891
upscaling inf time is  0.07278776168823242
t2 is  0.023417949676513672
t1 + t2 is  0.028955459594726562
whole is 0.10174322128295898
